In [118]:
# Import libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
# import import_ipynb
# Se ignoran los "FutureWarnings" molestos
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) 

In [119]:
# Read cvs file
DATASET_PATH = 'weatherAUS.csv'
dataset = pd.read_csv(DATASET_PATH)


In [120]:
# Visualize rows
dataset.head(145000)


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144995,2016-03-18,Uluru,23.0,33.6,9.4,NaN,NaN,ESE,37.0,SSE,...,76.0,43.0,1009.8,1006.6,8.0,7.0,25.3,32.8,Yes,No
144996,2016-03-19,Uluru,21.5,34.0,0.0,NaN,NaN,ESE,31.0,E,...,53.0,32.0,1012.3,1008.2,NaN,5.0,24.1,32.4,No,No
144997,2016-03-20,Uluru,22.5,35.9,0.0,NaN,NaN,S,35.0,ENE,...,30.0,32.0,1012.8,1008.3,NaN,NaN,26.5,34.4,No,No
144998,2016-03-21,Uluru,22.5,33.2,0.4,NaN,NaN,ENE,31.0,E,...,24.0,23.0,1013.1,1010.3,1.0,NaN,27.6,31.5,No,No


In [121]:
# Visualize dataset format
print('Dataset shape {}'.format(dataset.shape))

Dataset shape (145460, 23)


In [122]:
# Descriptive and statistics for each column
dataset.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,143975.000000,144199.000000,142199.000000,82670.000000,75625.000000,135197.000000,143693.000000,142398.000000,142806.000000,140953.000000,130395.00000,130432.000000,89572.000000,86102.000000,143693.000000,141851.00000
mean,12.194034,23.221348,2.360918,5.468232,7.611178,40.035230,14.043426,18.662657,68.880831,51.539116,1017.64994,1015.255889,4.447461,4.509930,16.990631,21.68339
std,6.398495,7.119049,8.478060,4.193704,3.785483,13.607062,8.915375,8.809800,19.029164,20.795902,7.10653,7.037414,2.887159,2.720357,6.488753,6.93665
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.50000,977.100000,0.000000,0.000000,-7.200000,-5.40000
25%,7.600000,17.900000,0.000000,2.600000,4.800000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.90000,1010.400000,1.000000,2.000000,12.300000,16.60000
50%,12.000000,22.600000,0.000000,4.800000,8.400000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.60000,1015.200000,5.000000,5.000000,16.700000,21.10000
75%,16.900000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.40000,1020.000000,7.000000,7.000000,21.600000,26.40000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.00000,1039.600000,9.000000,9.000000,40.200000,46.70000


In [123]:
# se calcula porcentaje de valores null por columna y se dropean las columnas con porcentaje de nan superior a 10%
#aca hay que verificar que cada columna dropeada no contiene valores excluyentes con algun otro dato -->estos pueden ser recuperados
#hace falta agregar un modulo que hace siguiente calculo: si Cloud9am y Cloud3pm son distintos de nan y Sunshine es nan, se reemplaza o se complementa
#se puede quitar las ultimas dos columnas donde se expresa literalmente si llovió, alla tenemos la columna de rainfall con milimitros.
# # pressure y evaporation no parecen tener sustitutos
per=[]
dataset_ = dataset.copy()
for column in dataset_:
   tot=dataset_[column].isnull().sum()
   per=tot/len(dataset[column])*100
   if per > 10:
    print(column,"porcentaje de Null",per,"dropea")
    dataset_ = dataset_.drop(column, 1)
   else:
    print(column,"cantidad de Null",tot,"porcentaje de Null",per, "se conserva")

Date cantidad de Null 0 porcentaje de Null 0.0 se conserva
Location cantidad de Null 0 porcentaje de Null 0.0 se conserva
MinTemp cantidad de Null 1485 porcentaje de Null 1.0208992162793895 se conserva
MaxTemp cantidad de Null 1261 porcentaje de Null 0.8669049910628353 se conserva
Rainfall cantidad de Null 3261 porcentaje de Null 2.2418534304963567 se conserva
Evaporation porcentaje de Null 43.166506256015396 dropea
Sunshine porcentaje de Null 48.00976213391998 dropea
WindGustDir cantidad de Null 10326 porcentaje de Null 7.098858792795271 se conserva
WindGustSpeed cantidad de Null 10263 porcentaje de Null 7.055547916953114 se conserva
WindDir9am cantidad de Null 10566 porcentaje de Null 7.263852605527293 se conserva
WindDir3pm cantidad de Null 4228 porcentaje de Null 2.906641000962464 se conserva
WindSpeed9am cantidad de Null 1767 porcentaje de Null 1.214766946239516 se conserva
WindSpeed3pm cantidad de Null 3062 porcentaje de Null 2.105046060772721 se conserva
Humidity9am cantidad de 

In [124]:
# Visualize dataset format
print('Dataset shape {}'.format(dataset_.shape))

Dataset shape (145460, 17)


In [125]:
# necesitamos pasar las variables categoricas a variables numericas ademas podemos convertir si y no a 1 y 0
dataset_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   WindGustDir    135134 non-null  object 
 6   WindGustSpeed  135197 non-null  float64
 7   WindDir9am     134894 non-null  object 
 8   WindDir3pm     141232 non-null  object 
 9   WindSpeed9am   143693 non-null  float64
 10  WindSpeed3pm   142398 non-null  float64
 11  Humidity9am    142806 non-null  float64
 12  Humidity3pm    140953 non-null  float64
 13  Temp9am        143693 non-null  float64
 14  Temp3pm        141851 non-null  float64
 15  RainToday      142199 non-null  object 
 16  RainTomorrow   142193 non-null  object 
dtypes: float64(10), object(7)
mem

##en caso de conservar las ultimas dos columnas, se puede reemplazar los nan usando knn 
#sobre columna de rainfall y posteriormente usar el siguiente comando
#dataset.replace({"Yes": 1, "No": 0})
#print("columna RainToday contiene: \n",pd.value_counts(dataset.RainToday),"columna RainTomorrow contiene: \n",pd.value_counts(dataset.RainTomorrow))

In [126]:
# Codificación de la categoría DATE en dos columnas: Year y MonthDay que es un número con decimales entre 1 y 13


if 'Date' in dataset_.columns:
    date = dataset_['Date'].str.split(pat='-', expand = True)
    date.shape
    year = np.uint16(date[0])
    month_day = np.float32(date[1]) + np.divide(np.float32(date[2]),32)
    dataset_ = dataset_.drop('Date', 1)
    dataset_.insert(0, 'Year', year)
    dataset_.insert(1, 'MonthDay', month_day)


In [127]:
# Codificación de la categoría 'Location'
import geolocators
import importlib
importlib.reload(geolocators)

# Se agregan las columnas nuevas
if 'Latitude' in dataset_.columns:
    print ('la columna latitude ya existe')
else:
    dataset_.insert(2, 'Latitude', np.empty(len(dataset_['Location'])))
if 'Longitude' in dataset_.columns:
    print ('la columna longitude ya existe')
else:
    dataset_.insert(3, 'Longitude', np.empty(len(dataset_['Location'])))

print('El dataset tiene {} localidades distintas'.format(dataset_['Location'].unique().shape[0]))
for loc in dataset_['Location'].unique():
    print(geolocators.coordinates(loc), loc)
    dataset_.Latitude.replace()
    dataset_['Latitude'] = dataset_['Latitude'].where(dataset_['Location'] != loc, geolocators.coordinates(loc)[0])
    dataset_['Longitude'] = dataset_['Longitude'].where(dataset_['Location'] != loc, geolocators.coordinates(loc)[1])
if 'Location' in dataset_.columns: dataset_ = dataset_.drop('Location', 1)




El dataset tiene 49 localidades distintas
(-11, 13) Albury
(-8, 17) BadgerysCreek
(-6, 12) Cobar
(-5, 20) CoffsHarbour
(-4, 16) Moree
(-7, 18) Newcastle
(-8, 18) NorahHead
(-4, 34) NorfolkIsland
(-8, 17) Penrith
(-12, 12) Richmond
(-8, 18) Sydney
(-8, 18) SydneyAirport
(-10, 14) WaggaWagga
(-7, 18) Williamtown
(-9, 17) Wollongong
(-10, 16) Canberra
(-10, 16) Tuggeranong
(-10, 15) MountGinini
(-12, 10) Ballarat
(-11, 11) Bendigo
(-13, 14) Sale
(-12, 11) MelbourneAirport
(-12, 11) Melbourne
(-9, 9) Mildura
(-10, 8) Nhil
(-13, 8) Portland
(-12, 12) Watsonia
(-12, 8) Dartmoor
(-2, 20) Brisbane
(8, 12) Cairns
(-3, 20) GoldCoast
(5, 13) Townsville
(-9, 5) Adelaide
(-12, 7) MountGambier
(-9, 5) Nuriootpa
(-6, 3) Woomera
(-10, -15) Albany
(-9, -17) Witchcliffe
(-6, -16) PearceRAAF
(-6, -17) PerthAirport
(-6, -17) Perth
(-7, -11) SalmonGums
(-9, -16) Walpole
(-17, 14) Hobart
(-16, 14) Launceston
(1, 0) AliceSprings
(12, -2) Darwin
(10, 0) Katherine
(0, -1) Uluru


In [142]:
#definimos encoder y normalizamos los datos (menos la fecha), generamos otra tabla:
import sklearn.preprocessing as preprocessing
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
enc = preprocessing.OrdinalEncoder()
dataset_scal_=dataset_.copy()
k=0

for column in dataset_:
    if np.dtype(np.dtype(dataset_[column])) == "object" and column not in ["Year","MonthDay"]:
       dataset_scal_.insert(k, f'{column}_enc_scal', ss.fit_transform(enc.fit_transform(dataset_[column].values.reshape(-1, 1))))
       dataset_scal_ = dataset_scal_.drop(column, 1)

    elif column not in ["Year","MonthDay"]:
       dataset_scal_.insert(k, f'{column}_scal', ss.fit_transform(dataset_[column].values.reshape(-1, 1)))
       dataset_scal_ = dataset_scal_.drop(column, 1)

    k=k+1
       


In [144]:
dataset_scal_

,Year,MonthDay,Latitude_scal,Longitude_scal,MinTemp_scal,MaxTemp_scal,Rainfall_scal,WindGustDir_enc_scal,WindGustSpeed_scal,WindDir9am_enc_scal,WindDir3pm_enc_scal,WindSpeed9am_scal,WindSpeed3pm_scal,Humidity9am_scal,Humidity3pm_scal,Temp9am_scal,Temp3pm_scal,RainToday_enc_scal,RainTomorrow_enc_scal
0,2008,12.03125,-0.624896,0.37362,0.188477,-0.045139,-0.207704,1.132740,0.291377,1.251859,1.355682,0.668126,0.605844,0.111365,-1.420435,-0.013968,0.016811,-0.537569,-0.537551
1,2008,12.06250,-0.624896,0.37362,-0.749247,0.263892,-0.278475,1.347220,0.291377,-0.288972,1.573040,-1.126533,0.378823,-1.307515,-1.276175,0.032266,0.377216,-0.537569,-0.537551
2,2008,12.09375,-0.624896,0.37362,0.110333,0.348173,-0.278475,1.561701,0.438360,1.251859,1.573040,0.555960,0.832864,-1.622822,-1.035742,0.617897,0.218638,-0.537569,-0.537551
3,2008,12.12500,-0.624896,0.37362,-0.467930,0.671251,-0.278475,-0.797586,-1.178453,0.371384,-1.687324,-0.341369,-1.096812,-1.254964,-1.708954,0.170969,0.694374,-0.537569,-0.537551
4,2008,12.15625,-0.624896,0.37362,0.829255,1.275266,-0.160523,1.132740,0.070902,-1.389565,-0.165821,-0.790034,0.151802,0.689427,-0.891482,0.124735,1.155693,-0.537569,-0.537551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017,6.65625,1.281150,-0.81984,-1.468168,0.025095,-0.278475,-1.655509,-0.664013,0.371384,-1.469966,-0.117037,-0.869791,-0.939657,-1.324262,-1.061938,0.103308,-0.537569,-0.537551
145456,2017,6.68750,1.281150,-0.81984,-1.343138,0.291985,-0.278475,-0.368625,-1.325436,0.371384,-1.035251,-0.117037,-1.096812,-0.676902,-1.468521,-0.938648,0.406049,-0.537569,-0.537551
145457,2017,6.71875,1.281150,-0.81984,-1.061821,0.516735,-0.278475,-1.012067,-0.223064,0.371384,1.355682,-0.565702,-1.096812,-0.834555,-1.324262,-0.692066,0.636709,-0.537569,-0.537551
145458,2017,6.75000,1.281150,-0.81984,-0.686732,0.530782,-0.278475,0.274817,-0.884487,0.591503,-1.035251,-0.117037,-1.323832,-0.939657,-1.324262,-0.291372,0.622292,-0.537569,-0.537551
